In [102]:
!pip install git+https://github.com/stanfordLINQS/SQcircuit.git@dev-ab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/stanfordLINQS/SQcircuit.git (to revision dev-ab) to /tmp/pip-req-build-5k9mw8dx
  Running command git clone -q https://github.com/stanfordLINQS/SQcircuit.git /tmp/pip-req-build-5k9mw8dx
  Running command git checkout -b dev-ab --track origin/dev-ab
  Switched to a new branch 'dev-ab'
  Branch 'dev-ab' set up to track remote branch 'dev-ab' from 'origin'.


In [134]:
import SQcircuit as sq
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

In [135]:
loop1 = sq.Loop()

In [141]:
# define the circuit elements
# C = sq.Capacitor(5, 'GHz', Q=1e6)
# L = sq.Inductor(2, 'GHz', loops = [loop1])
C = sq.Capacitor(7.746, 'pF', Q=1e6)
L = sq.Inductor(81.67, 'uH', loops = [loop1])

# define the circuit
elements = {
    (0, 1): [C, L],
}

cr = sq.Circuit(elements)

In [142]:
cr.description()

H = ω_1ad_1a_1 + EL_1(2265563510.186φ_1)(φ_e1) 
------------------------------------------------------------
mode 1:       harmonic       φ_1 = zp_1(a_1+ad_1)       ω_1/2π = 0.00633       zp_1 = 5.55e-10
------------------------------------------------------------
parameters:       EL_1 = 0.002       
loops:            φ_e1/2π = 0.0       


In [143]:
cr.set_trunc_nums([10,])
cr.diag(2)

(array([-2.98805608e-17,  6.32776158e-03]),
 [Quantum object: dims = [[10], [1]], shape = (10, 1), type = ket
  Qobj data =
  [[ 7.35401504e-01+6.77631631e-01j]
   [-8.32667268e-17-5.55111512e-17j]
   [-2.77555756e-16-6.93889390e-18j]
   [-2.77555756e-16-1.94289029e-16j]
   [ 1.11022302e-16+1.38777878e-16j]
   [ 5.55111512e-17-1.66533454e-16j]
   [-2.77555756e-17+5.55111512e-17j]
   [-2.77555756e-17-2.77555756e-17j]
   [ 1.04083409e-17+1.11022302e-16j]
   [-5.55111512e-17+1.11022302e-16j]],
  Quantum object: dims = [[10], [1]], shape = (10, 1), type = ket
  Qobj data =
  [[ 1.34186372e-16+5.42040893e-16j]
   [ 6.61144442e-02+9.97812047e-01j]
   [-3.19189120e-16-1.66533454e-16j]
   [ 3.98986399e-17+4.33680869e-17j]
   [-4.16333634e-17+2.08166817e-17j]
   [ 1.24900090e-16-2.49800181e-16j]
   [ 5.55111512e-17-5.55111512e-17j]
   [ 1.66533454e-16-8.32667268e-17j]
   [-8.32667268e-17+4.99600361e-16j]
   [-2.22044605e-16+2.22044605e-16j]]])

In [130]:
cr._efreqs
print(f"Fundamental resonant frequency: {cr._efreqs[1] - cr._efreqs[0]} Hz")

Fundamental resonant frequency: 39758498.58406887 Hz


In [131]:
cr._evecs

[Quantum object: dims = [[10], [1]], shape = (10, 1), type = ket
 Qobj data =
 [[-2.45603957e-01-9.69370258e-01j]
  [ 7.77156117e-16+3.98986399e-17j]
  [-3.88578059e-16-3.33066907e-16j]
  [-7.80625564e-18+3.33066907e-16j]
  [ 5.55111512e-17+1.38777878e-17j]
  [ 0.00000000e+00-5.55111512e-17j]
  [ 2.77555756e-17-3.46944695e-18j]
  [ 5.55111512e-17+0.00000000e+00j]
  [ 2.77555756e-16+5.55111512e-17j]
  [-5.55111512e-17+2.77555756e-17j]],
 Quantum object: dims = [[10], [1]], shape = (10, 1), type = ket
 Qobj data =
 [[ 5.38675592e-16-1.53622782e-16j]
  [-7.72467852e-02-9.97012003e-01j]
  [ 4.99600361e-16-7.07767178e-16j]
  [ 8.71698547e-17-2.07270211e-32j]
  [ 8.23993651e-17-1.18828558e-16j]
  [-3.29597460e-16-1.38777878e-17j]
  [-2.87524253e-33+2.25514052e-17j]
  [-3.81702691e-33-1.11022302e-16j]
  [-3.61746937e-33-1.66533454e-16j]
  [ 1.15508595e-32+1.35591517e-32j]]]

In [144]:
res_grad_C = cr._get_partial_omega(C, 1, subtract_ground = True)
partial_H = cr._get_partial_H(C, None)
e_0, e_1 = cr._evecs[0], cr._evecs[1]
a = e_0.dag() * (partial_H * e_0)
b = e_1.dag() * (partial_H * e_1)
print(res_grad_C)
print((b - a).data[0, 0].real)
print(a, b)
print(C.value())

-2.5663890093429473e+18
-2.5663890093429473e+18
Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[-1.2831945e+18+51.94836587j]] Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[-3.84958351e+18-11.87838961j]]
7.746e-12


In [145]:
omega_target = 20e6 # 20 MHz
learning_rate = 10**(-2.8)
for step in range(100):
  cr.set_trunc_nums([10,])
  cr.diag(2)
  omega = cr._efreqs[1] - cr._efreqs[0]
  print(f"Resonant frequency: {omega}")
  ## Backprop C
  C_grad_omega = cr._get_partial_omega(C, 1, subtract_ground = True) * 1.0545718e-34
  C_grad = C_grad_omega * 2 * (omega - omega_target)
  new_C_value = C.cValue * 1e-12 - C_grad * learning_rate
  print(f"C grad: {C_grad}")
  print(f"C value: {C.value()} -> {new_C_value}")

  ## Backprop L
  # print(f"L grad: {cr._get_partial_omega(L, 1, subtract_ground = True)}")
  # print(f"L: {(cr._memory_ops['ind_hamil'][(L, 0)] / L.value()**2) / np.sqrt(1.0545718e-34)}")
  '''L_grad_omega = cr._get_partial_omega(L, 1, subtract_ground = True)# * 1.0545718e-34
  L_grad = L_grad_omega * 2 * (omega - omega_target)
  new_L_value = L.lValue * 1e-6 - L_grad * learning_rate
  print(f"L grad: {L_grad}")
  print(f"L value: {L.value()} -> {new_L_value}")'''

  C = sq.Capacitor(new_C_value * 1e12, 'pF', Q=1e6)
  # L = sq.Inductor(new_L_value * 1e6, 'uH', loops = [loop1])

  # define the circuit
  elements = {
      (0, 1): [C, L],
  }

  cr = sq.Circuit(elements)

Resonant frequency: 39758498.5840689
C grad: -1.0695044018561999e-08
C value: 7.746e-12 -> 2.4696502458090868e-11
Resonant frequency: 22266450.722458966
C grad: -2.154953871179479e-10
C value: 2.4696502458090868e-11 -> 2.5038039630140876e-11
Resonant frequency: 22114063.93518558
C grad: -1.969076138842785e-10
C value: 2.5038039630140876e-11 -> 2.535011716692983e-11
Resonant frequency: 21977522.666694075
C grad: -1.8079915212113767e-10
C value: 2.535011716692983e-11 -> 2.5636664512329365e-11
Resonant frequency: 21854353.420713995
C grad: -1.6670364863430796e-10
C value: 2.5636664512329365e-11 -> 2.590087199020831e-11
Resonant frequency: 21742602.663218707
C grad: -1.5426651584935772e-10
C value: 2.590087199020831e-11 -> 2.614536794100265e-11
Resonant frequency: 21640701.922328427
C grad: -1.43212994512366e-10
C value: 2.614536794100265e-11 -> 2.637234524107727e-11
Resonant frequency: 21547373.804985404
C grad: -1.333266563106862e-10
C value: 2.6372345241077267e-11 -> 2.6583653751037677e

In [121]:
# get_partial_H for L 0 since Phi 0; need to address this
print(cr._get_external_flux_at_element(None))

[[0.]]
